# ロジスティック回帰と回帰係数

In [1]:
import warnings
warnings.filterwarnings('ignore')

from utils import datasource, helper

query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX imas: <https://sparql.crssnky.xyz/imasrdf/URIs/imas-schema.ttl#>
SELECT ?label
WHERE{
  ?s rdf:type imas:Idol;
  rdfs:label ?label;
  imas:Title ?title.
  FILTER (regex(?title, "765AS") || regex(?title, "MillionStars"))
}
"""

# ニコニコ大百科からのデータ取得
df_idols = datasource.imasparql(query)
df_niconico = datasource.niconico_dic(df_idols['label'].values)

import pandas as pd
pd.set_option("display.max_colwidth", 100)

df = pd.DataFrame({
    'label': df_idols['label'].values,
    'text': df_niconico['text']
})
df.head()

,label,text
0,萩原雪歩,ニコニコ大百科\n \n萩原雪歩（はぎわら ゆきほ）は、バンダイナムコエンターテインメントのゲーム『アイドルマスター』（The Idolm@ster）シリーズ...
1,永吉昴,ニコニコ大百科\n \n「永吉昴」（ながよし すばる）とは、バンダイナムコエンターテインメントのゲーム『アイドルマスター ミリオンライブ！』シリーズの登場人物...
2,木下ひなた,ニコニコ大百科\n \n「木下ひなた」（きのした ひなた）とは、バンダイナムコエンターテインメントのゲーム『アイドルマスター ミリオンライブ！』シリーズの登場...
3,伊吹翼,ニコニコ大百科\n \n「伊吹翼」（いぶき つばさ）とは、バンダイナムコエンターテインメントのゲーム『アイドルマスター ミリオンライブ！』シリーズの登場人物で...
4,白石紬,ニコニコ大百科\n \n「白石紬」（しらいし つむぎ）とは、バンダイナムコエンターテインメントの『アイドルマスター ミリオンライブ！』シリーズの登場アイドルで...


In [2]:
# 形態素解析
from janome.tokenizer import Tokenizer
tokenizer = Tokenizer()
tokenize = helper.tokenize_func(tokenizer)
df['tokenized'] = df['text'].apply(tokenize)

# Word2Vecモデルの準備
w2v_model = helper.w2v_pretrained_model()
word_list = w2v_model.wv.index2word

# TF-IDFの計算
df['tfidf'] = helper.tfidf_wordlist(df['tokenized'], word_list)

# 分散表現の座標に変換
embedding_matrix = w2v_model[word_list]
def embedding_product(e):
    return (e * embedding_matrix)[0]
df['embedding'] = df['tfidf'].apply(embedding_product)

# 並べ替え(アイドル順)
df_id = datasource.idol_id()
df_orderd = df_id.merge(df, left_on="label", right_on="label").sort_values("id")
df_orderd[['label', 'embedding']].head()

,label,embedding
0,天海春香,"[-2.5092840439978947, 0.5338919582236741, 3.253890018920983, -1.2640430555630706, -1.24759669638..."
1,如月千早,"[-0.9617702611653435, 0.749174542566589, 2.746178093471357, -0.25107552697441265, 0.407746981821..."
2,星井美希,"[-0.35180219343157654, 0.9926401922258702, 2.1392033760306144, -0.7489018767090513, -1.133355244..."
3,萩原雪歩,"[-0.4736889503707105, 1.3252699915609691, 2.8868200450541366, -1.0935365266558839, -0.3392575369..."
4,高槻やよい,"[-1.5981685811522082, -0.19402577814063005, 3.7298617915149306, -0.9640515292948152, 0.426477972..."


In [3]:
import numpy as np
from sklearn.linear_model import LogisticRegression

# 特徴量行列の準備
x = np.array(list(df_orderd['embedding'].values))

# ロジスティック回帰の実行
for idx in range(0,52):
    targets = [25, idx]
    y = np.zeros(x.shape[0])
    y[targets] = 1
    lr = LogisticRegression(multi_class='auto', solver='lbfgs')
    model = lr.fit(x, y)
    word = w2v_model.most_similar([model.coef_[0]], [], 15)
    print('{}: {}'.format('x'.join(df_id['label'][targets]) ,','.join([x[0] for x in word])))

七尾百合子x天海春香: 幻想,怪奇,少女,魅惑,ものの,ファンタジー,名作,憂鬱,紀行,童話,パロディ,作品,悲劇,妖精,空想
七尾百合子x如月千早: 事件,幻想,ノンフィクション,カルテ,悲劇,ルポルタージュ,エッセイ,カフカ,このことは,のなかで,紀行,ボードレール,それは,世界,試論
七尾百合子x星井美希: 上梓,シナリオ,連載,考察,改題,バトンタッチ,タイムトラベル,ログアウト,ノンフィクション,公刊,タイムスリップ,脚色,刊行,論考,幻想
七尾百合子x萩原雪歩: レイプ,妄想,連載,怪奇,コンテンツアイコン,ショートストーリー,少女,エッセイ,草紙,日記,改題,エピソード,ノベルス,ものを,事件
七尾百合子x高槻やよい: 幻想,カルテ,妄想,うつ,日記,わが,たとえば,コンテンツアイコン,シュレーディンガー,コンテンツアイコンセクシャル,忘却,であるとしている,のうた,極性,著者
七尾百合子x菊地真: 少女,ジェンダー,幻想,コンテンツアイコンセクシャル,心理,妄想,オカルト,思考,情念,セクシャル,探究,レイプ,思想,恋,コンテンツアイコン
七尾百合子x水瀬伊織: 友人,著者,作家,知人,著書,アルセーヌルパン,エッセイ,日記,ガールフレンド,少女,富豪,三島由紀夫,ジャーナリスト,家族,事件
七尾百合子x四条貴音: 少女,架空,妖怪,コンテンツアイコン,謎,妖精,迷宮,偉人,奇譚,年代記,少年,一群,幻想,怪事,紀行
七尾百合子x秋月律子: カルテ,レポート,少女,情報,テクノロジー,少年,論文,カウンセリング,サイバー,送致,最新,コンピュータウイルス,傍受,構築,ハッキング
七尾百合子x三浦あずさ: 少女,あずさ,アフタヌーン,ビッグコミック,ゆり,少年,ビッグコミックスピリッツ,恋,マーガレット,先生,とき,エッセイ,なかよし,芙,とゆめ
七尾百合子x双海亜美: 刊行,ファイル,新編,カルテ,をまとめたもの,簿,月報,事件,にまとめて,ものがたり,出版,上梓,ほかを,推理,をまとめた
七尾百合子x双海真美: 軌跡,アトランティス,幻想,宇宙,よみがえる,時空,銀河,クトゥルフ,地底,ウルク,月面,灼熱,ディンギル,ゾル,せよ
七尾百合子x我那覇響: 出版,週刊,アニメディア,ムック,別冊,月刊,先生,著者,メディアファクトリー,月報,日記,マガ